In [ ]:
def D(k,s,n,R = QQ):
    return identity_matrix(n) + matrix(R, n, n, lambda i, j: s - 1 if (i == k and j == k) else 0)

def E(k,l,s,n,R = QQ):
    return identity_matrix(n) + matrix(R, n, n, lambda i, j: s if (i == k and j == l) else 0)

def P(k,l,n,R = QQ):
    P = identity_matrix(n)
    P[k,k] = 0
    P[l,l] = 0
    P[k,l] = 1
    P[l,k] = 1
    return P



# aufgepasst, M muss exacte Eintraege haben wegen M.eigenvectors_right()
# D.h., keine floats oder doubles aber z.B. QQ oder SR

def uppertriangulate(M, depth=0, loquacious=False):
    
    if loquacious and depth == 0:
        print('Rekusive Funktion --uppertriangulate-- initiiert', end='\n\n')
        print(M, '= Matrix in' , end='\n\n')


    if M.nrows() != M.ncols():
        print('matrix not square')
        return
    
    n = M.nrows()
    R = M.base_ring()
    if n == 1:
        return [identity_matrix(1), M]
    
    columns = [M.eigenvectors_right()[0][1][0]]
    I = identity_matrix(n)
    
    for i in range(n):
        columnscopy = columns.copy()
        columnscopy.append(vector(R,I[:,i].list()))
        if column_matrix(R, columnscopy).transpose().kernel().dimension() == 0:
            columns.append(vector(R,I[:,i].list()))
        i = i + 1
        
    Q = column_matrix(R, columns)

    Qtot = Q * matrix.block_diagonal(identity_matrix(1), uppertriangulate((~Q*M*Q)[1:,1:], depth + 1, loquacious)[0])
    
    if loquacious:
        print('-------------------------')
        print('Rekursionstiefe = ', depth, end='\n\n')
        print(Q, '= Q', end='\n\n')
        print(~Qtot * M * Qtot, ' = M', end=' ')
        
        if depth == 0:
            print('= Matrix out', 'Rekursive Funktion --uppertriangulate-- geendet', sep='\n\n', end='\n\n')
        else:
            print('\n')
    
    return [Qtot, ~Qtot * M * Qtot]
    
    
    
    
    
# in dieser Funktion muss M eine obere Dreiecksmatrix sein

def groupeigenvalues(M, loquacious=False):
    
    if loquacious:
        print('Funktion --groupeigenvalues-- initiiert', end='\n\n')
        print(M, '= Matrix in', end='\n\n')
    
    if M.nrows() != M.ncols():
        print('matrix not square')
        return
    n = M.nrows()
    
    for i in range(n):
        for j in range(i+1,n):
            if M[j,i] != 0:
                print('matrix not uppertriangular')
                return
    
    R = M.base_ring()
    
    Qtot = matrix.identity(n)
    
    for i in range(n - 2):
        
        j = 1
        foundsameeigenvalue = true
        
        while j < n - i and M[i,i] != M[i + j, i + j]:
            j = j + 1
            if j == n - i - 1 and M[i,i] != M[i + j, i + j]:
                foundsameeigenvalue = false
                
        if j > 1 and foundsameeigenvalue:
            for k in range(j - 1):
                a = i + j - k - 1
                b = i + j - k
                Q1 = E(a, b, M[a, b]/(M[b, b] - M[a, a]), n, R)
                Q2 = P(a, b, n ,R)
                Q = E(a, b, M[a, b]/(M[b, b] - M[a, a]), n, R) * P(a, b, n ,R)
                
                M = ~Q * M * Q
                
                Qtot = Qtot * Q
                
                if loquacious:
                    print('-------------------------')
                    print( Q1, ' *\n\n', Q2,' = Q', sep='', end='\n\n')
                    print(M, ' = M', end='')
                    if i == n - 3:
                        print(' = Matrix out', end='\n\n')
                    else:
                        print('\n')
                
    if loquacious:
        print('Funktion --groupeigenvalues-- geendet', end='\n\n')
    return [Qtot, M]





# in dieser Funktion muss M eine obere Dreiecksmatrix sein,
# und alle gleichen Eigenwerte von M mussen neben einander auf der Diagonale stehen

def blockdiagonalize(M, loquacious=False):
    
    if loquacious:
        print('Funktion --blockdiagonalize-- initiiert', end='\n\n')
        print(M, '= Matrix in', end='\n\n')
    
    if M.nrows() != M.ncols():
        print('matrix not square')
        return
    n = M.nrows()
    
    for i in range(n):
        for j in range(i+1,n):
            if M[j,i] != 0:
                print('matrix subtriangular')
                return
            
    R = M.base_ring()
    
    blocksizes = []
    
    k = 1
    for i in range(n - 1):
        if M[i,i] == M[i + 1, i + 1]:
            k = k + 1
        else:
            blocksizes.append(k)
            k = 1
    blocksizes.append(k)
    
    Qtot = matrix.identity(n)
    
    blocksizesum = 0
    for blocksize in blocksizes:
        
        for j in range(blocksize):
            for i in range(blocksizesum):
                k = blocksizesum - i - 1
                l = j + blocksizesum 
                
                if M[k,l] != 0 or [k,l] == [0, n - 1]:
                
                    Q = E(k, l, M[k,l]/(M[l,l] - M[k,k]), n, R)
                    M = ~Q * M * Q

                    Qtot = Qtot * Q

                    if loquacious:
                        print('-------------------------')
                        print(Q, '= Q', end='\n\n')
                        print(M, ' = M', end='')
                        if [k,l] == [0, n - 1]:
                            print(' = Matrix out', end='\n\n')
                        else:
                            print('\n')
                else:
                    if loquacious:
                        print('-------------------------')
                        print('Schritt übergehen')
        
        blocksizesum = blocksizesum + blocksize
        
    if loquacious:
        print('Funktion --blockdiagonalize-- geendet', end='\n\n')
        
    return [Qtot, M, blocksizes]






# in dieser Funktion muss M eine obere Dreiecksmatrix sein,
# mit nur einem Eigenwert

def partitionblock(M, depth=0, loquacious=False):
    
    if loquacious and depth == 0:
        print('Rekusive Funktion --partitionblock-- initiiert', end='\n\n')
        print(M, '= Matrix in' , end='\n\n')
        
    if M.nrows() != M.ncols():
        print('matrix not square')
        return
    n = M.nrows()
    
    R = M.base_ring()
    
    if n == 1:
        if loquacious:
            print('-------------------------')
            print('Rekursionstiefe = ', depth, end='\n\n')
            print(matrix.identity(1), '= Q', end='\n\n')
            print(M, ' = M', end='\n\n')
            
        return [matrix.identity(1), M]

    Qtot = matrix.block_diagonal(partitionblock(M[0:n - 1, 0:n - 1], depth + 1, loquacious)[0], matrix.identity(1))
    M = ~Qtot * M * Qtot
    
    if loquacious:
        print('-------------------------')
        print('Rekursionstiefe = ', depth, end='\n\n')
    
    
    # alle Eintraege in der letzten Spalte, die nicht Null sind und rechts neben ein einz stehen,
    # werden mittels E matrizen Null gemacht.
    
    for i in range(n - 2):
        if M[i, i+1] == 1 and M[i, n - 1] != 0:
            
            Q = E(i+1, n - 1, -M[i,n-1], n, R)
            M = ~Q * M * Q
            
            Qtot = Qtot * Q
            
            if loquacious:
                print('-------------------------')
                print(Q, '= Q', end='\n\n')
                print(M, ' = M', end='\n\n')
    
    # l wird bestimmt, sodass M_{ln} der Eintrag in der letzen Spalten, die nicht Null ist 
    # und am naehsten an der Diagonale steht, aber nicht ein Diagonaleintrag ist
    
    l = -1
    
    for i in range(n - 1):
        if M[i, n - 1] != 0:
            l = i
    
    if l != -1 and M[l, n - 1] != 1:
        Q = D(n - 1, 1/M[l, n - 1], n, R)
        M = ~Q * M * Q
        if loquacious:
            print('-------------------------')
            print(Q, '= Q', end='\n\n')
            print(M, ' = M', end='\n\n')
        Qtot = Qtot * Q
    
    # alle uebrige nichtnull Eintraege der letzten Spalten, die nich auf die Diagonale stehen,
    # werden bestimmt und null gemacht
    
    ks = []
    
    for i in range(l):
        if M[i, n - 1] != 0:
            ks.append(i)
    
    for k in ks:
        
        ktemp = k
        ltemp = l
        e = M[k,n - 1]
        
        while(e != 0):
            
            Q = E(ktemp,ltemp,e,n,R)
            M = ~Q * M * Q
            
            if loquacious:
                print('-------------------------')
                print(Q, '= Q', end='\n\n')
                print(M, ' = M', end='\n\n')
            
            Qtot = Qtot * Q
            
            e = M[ktemp - 1, ltemp]
            
            ktemp = ktemp - 1
            ltemp = ltemp - 1
    
    if l != n - 2 and l != -1:
        
        p = 1
        
        if l != 0:
            i = l - 1
            while M[i, i + 1] == 1:
                p = p + 1

                if i == 0:
                    break

                else:
                    i = i - 1
                    
        q = 1    
        
        j = n - 3
        while M[j, j + 1] == 1:
            q = q + 1

            if j == 0:
                break

            else:
                j = j - 1
        
        P = matrix.block(R, [[matrix.identity(l - p + 1),0,0,0,0],[0,0,0,matrix.identity(p),0],[0,0,matrix.identity(n - 2 - q - l),0,0],[0,matrix.identity(q),0,0,0],[0,0,0,0,matrix.identity(1)]], subdivide=False)
        M = ~P * M * P
        
        if loquacious:
            print('-------------------------', p, q, l)
            print(P, '= P', end='\n\n')
            print(M, '= M', end='\n\n')
            
        Qtot = Qtot * P
                
        
        if p + 1 < q:
            P = matrix.block(R, [[matrix.identity(l - p + 1),0,0,0],[0,0,0,matrix.identity(q)],[0,0,matrix.identity(n - 2 - q - l),0],[0,matrix.identity(p + 1),0,0]], subdivide=False)
            M = ~P * M * P
            
            if loquacious:
                print('-------------------------')
                print(P, '= P', end='\n\n')
                print(M, '= M', end='\n\n')
                
            Qtot = Qtot * P
            
        if p != q and n - 2 - q - l > 0:

            blocksize = p + 1
        
            rightadjecentblocksize = p
            
            index = l + blocksize

            while index < n - 1 and M[index, index + 1] == 1:
                rightadjecentblocksize = rightadjecentblocksize + 1

                index = index + 1
                    
            while rightadjecentblocksize < blocksize:
                P = matrix.block(R, [[matrix.identity(index - blocksize),0,0,0],[0,0,matrix.identity(rightadjecentblocksize),0],[0,matrix.identity(blocksize),0,0],[0,0,0,matrix.identity(n - index - rightadjecentblocksize)]], subdivide=False)
                M = ~P * M * P
                
                if loquacious:
                    print('-------------------------')
                    print(P, '= P', end='\n\n')
                    print(M, ' = M', end='\n\n')
            
                Qtot = Qtot * P
                
                index = index + rightadjecentblocksize
                blocksize = rightadjecentblocksize
                
                while index < n - 1 and M[index, index + 1] == 1:
                    rightadjecentblocksize = rightadjecentblocksize + 1

                    index = index + 1
                        
    if l == -1:
        P = matrix.block(R, [[0,matrix.identity(n - 1)],[matrix.identity(1),0]], subdivide=False)
        M = ~P * M * P

        if loquacious:
            print('-------------------------')
            print(P, '= P', end='\n\n')
            print(M, ' = M', end='\n\n')

        Qtot = Qtot * P
        
                        
    return [Qtot, M]
    

def jordannormalform(M, loquacious=False):
    
    
    [Qtot, M] = uppertriangulate(M, 0, loquacious)
    
    [Q, M] = groupeigenvalues(M, loquacious)
    Qtot = Qtot * Q
    
    [Q, M, blocksizes] = blockdiagonalize(M, loquacious)
    Qtot = Qtot * Q
    
    s = 0
    sums = []
    blocks = []
    Qs = []
    for size in blocksizes:
        [Qpart, Mpart] = partitionblock(M[s:s + size,s:s + size], 0, loquacious)
        
        blocks.append(Mpart)
        Qs.append(Qpart)
        
        s = s + size
        sums.append(s)
        
    M = matrix.block_diagonal(blocks, subdivide=False)
    
    Q = matrix.block_diagonal(Qs, subdivide=False)
    Qtot = Qtot * Q
    
    return [Qtot, M]

In [ ]:
M = matrix(QQ, [[1,0,0,0,0,0],[0,5,1,0,0,0],[0,0,5,0,0,0],[0,0,0,1,1,0],[0,0,0,0,1,1],[0,0,0,0,0,1]])
R = matrix.random(ZZ, 6)
MC = ~R * M * R
MC

In [ ]:
[Q, J] = jordannormalform(MC, True)
J

In [189]:
Q*J*~Q - MC

[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]
[0 0 0 0 0 0]

In [190]:
mm = matrix(QQ, [[2,0,0,0,0,0,], [0,1,0,0,0,0],[0,0,1,986756,0,0],[0,0,0,2,0,0],[0,0,0,0,2,0],[0,1,0,0,0,1]])
jordannormalform(mm)

[
[       0        0        1        0        0        0]  [2 0 0 0 0 0]
[       0        0        0        0        0        1]  [0 2 0 0 0 0]
[       0        1        0        1        0        0]  [0 0 2 0 0 0]
[       0 1/986756        0        0        0        0]  [0 0 0 1 0 0]
[       1        0        0        0        0        0]  [0 0 0 0 1 1]
[       0        0        0        0        1        0], [0 0 0 0 0 1]
]